In [1]:
import requests
import nba_api
import numpy as np
import pandas as pd
import re


In [2]:
data = pd.read_csv('final_data.csv', index_col=0)
data

,GAME_ID,SCOREMARGIN,SCORE,TIME_REMAINING,PERIOD,TEAM_1,TEAM_2,PLAYER_1.1,PLAYER_1.2,PLAYER_1.3,PLAYER_1.4,PLAYER_1.5,PLAYER_2.1,PLAYER_2.2,PLAYER_2.3,PLAYER_2.4,PLAYER_2.5
0,22400061,NaN,NaN,0,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday
1,22400061,NaN,NaN,0,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday
2,22400061,3,0 - 3,12,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday
3,22400061,1,2 - 3,33,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday
4,22400061,NaN,NaN,44,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124987,22400039,29,98 - 127,694,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones
124988,22400039,NaN,NaN,703,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones
124989,22400039,28,99 - 127,703,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones
124990,22400039,27,100 - 127,703,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones


In [3]:
data.loc[0, 'SCOREMARGIN'] = 0
for i in range(1, 124992):
    if pd.isna(data.loc[i, 'SCOREMARGIN']) or data.loc[i, 'SCOREMARGIN'] == 0:
        data.loc[i, 'SCOREMARGIN'] = data.loc[i - 1, 'SCOREMARGIN']
    if data.loc[i, 'PERIOD'] == 1 and data.loc[i-1, 'PERIOD'] == 4:
        data.loc[i, 'SCOREMARGIN'] = 0
data.loc[data['SCOREMARGIN'] == "TIE", 'SCOREMARGIN'] = 0
data

,GAME_ID,SCOREMARGIN,SCORE,TIME_REMAINING,PERIOD,TEAM_1,TEAM_2,PLAYER_1.1,PLAYER_1.2,PLAYER_1.3,PLAYER_1.4,PLAYER_1.5,PLAYER_2.1,PLAYER_2.2,PLAYER_2.3,PLAYER_2.4,PLAYER_2.5
0,22400061,0,NaN,0,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday
1,22400061,0,NaN,0,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday
2,22400061,3,0 - 3,12,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday
3,22400061,1,2 - 3,33,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday
4,22400061,1,NaN,44,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124987,22400039,29,98 - 127,694,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones
124988,22400039,29,NaN,703,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones
124989,22400039,28,99 - 127,703,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones
124990,22400039,27,100 - 127,703,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones


In [4]:
data.loc[0, 'SCORE'] = "0 - 0"
for i in range(1, 124992):
    if pd.isna(data.loc[i, 'SCORE']) or data.loc[i, 'SCORE'] == "0 - 0":
        data.loc[i, 'SCORE'] = data.loc[i - 1, 'SCORE']
    if data.loc[i, 'PERIOD'] == 1 and data.loc[i-1, 'PERIOD'] == 4:
        data.loc[i, 'SCORE'] = "0 - 0"
        
data

,GAME_ID,SCOREMARGIN,SCORE,TIME_REMAINING,PERIOD,TEAM_1,TEAM_2,PLAYER_1.1,PLAYER_1.2,PLAYER_1.3,PLAYER_1.4,PLAYER_1.5,PLAYER_2.1,PLAYER_2.2,PLAYER_2.3,PLAYER_2.4,PLAYER_2.5
0,22400061,0,0 - 0,0,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday
1,22400061,0,0 - 0,0,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday
2,22400061,3,0 - 3,12,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday
3,22400061,1,2 - 3,33,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday
4,22400061,1,2 - 3,44,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124987,22400039,29,98 - 127,694,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones
124988,22400039,29,98 - 127,703,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones
124989,22400039,28,99 - 127,703,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones
124990,22400039,27,100 - 127,703,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones


In [5]:
SCORE_1 = []
SCORE_2 = []
for i in range(124992):
    match = re.match(r"(\d+)\s*-\s*(\d+)", data.loc[i, 'SCORE'])
    if match:
        SCORE_1.append(int(match.group(1)))
        SCORE_2.append(int(match.group(2)))
data['SCORE_1'] = SCORE_1
data['SCORE_2'] = SCORE_2

In [6]:
data.drop(columns=['SCORE'], inplace=True)

In [19]:
result = []
for i in range(124991):
    if data.loc[i, 'PERIOD'] == 4 and (data.loc[i+1, 'PERIOD'] == 1):
        if data.loc[i, 'SCORE_1'] > data.loc[i, 'SCORE_2']:
            result.append(data.loc[i, 'TEAM_1'])
        else:
            result.append(data.loc[i, 'TEAM_2'])
if data.loc[124991, 'SCORE_1'] > data.loc[124991, 'SCORE_2']:
    result.append(data.loc[124991, 'TEAM_1'])
else:
    result.append(data.loc[124991, 'TEAM_2'])
result

['BOS',
 'LAL',
 'IND',
 'ATL',
 'ORL',
 'MIL',
 'CLE',
 'CHA',
 'NOP',
 'MEM',
 'GSW',
 'BOS',
 'DAL',
 'OKC',
 'MIN',
 'ORL',
 'TOR',
 'ATL',
 'CLE',
 'NYK',
 'HOU',
 'CHI',
 'GSW',
 'LAL',
 'NOP',
 'LAC',
 'MIA',
 'BOS',
 'CLE',
 'OKC',
 'MEM',
 'MIN',
 'SAS',
 'PHX',
 'LAL',
 'BKN',
 'POR',
 'OKC',
 'LAC',
 'ORL',
 'WAS',
 'BOS',
 'MIA',
 'CLE',
 'CHI',
 'HOU',
 'DAL',
 'PHX',
 'SAC',
 'DAL',
 'SAC',
 'GSW',
 'CHA',
 'CLE',
 'DET',
 'WAS',
 'NYK',
 'CHI',
 'BKN',
 'OKC',
 'GSW',
 'POR',
 'MEM',
 'HOU',
 'SAS',
 'PHX',
 'BOS',
 'CLE',
 'NYK',
 'SAC',
 'BKN',
 'LAL',
 'NOP',
 'MIN',
 'OKC',
 'BOS',
 'MEM',
 'CLE',
 'SAS',
 'MIA',
 'DEN',
 'PHX',
 'OKC',
 'DET',
 'ATL',
 'DAL',
 'CLE',
 'GSW',
 'DET',
 'BOS',
 'BKN',
 'SAC',
 'UTA',
 'HOU',
 'MIN',
 'OKC',
 'POR',
 'IND',
 'DEN',
 'PHX',
 'LAC',
 'CHA',
 'IND',
 'ATL',
 'GSW',
 'HOU',
 'MEM',
 'CLE',
 'DAL',
 'DEN',
 'PHX',
 'LAC',
 'SAC',
 'MIN',
 'MIL',
 'SAS',
 'CHA',
 'DET',
 'ORL',
 'CLE',
 'NYK',
 'PHX',
 'MEM',
 'OKC',
 'MIN',


In [20]:
j = 0
data['WINNER'] = pd.Series()
for i in range(124991):
    data.loc[i, 'WINNER'] = result[j]
    if data.loc[i, 'PERIOD'] == 4 and (data.loc[i+1, 'PERIOD'] == 1):
        j+=1
data.loc[124991, "WINNER"] = result[j]
data

,GAME_ID,SCOREMARGIN,TIME_REMAINING,PERIOD,TEAM_1,TEAM_2,PLAYER_1.1,PLAYER_1.2,PLAYER_1.3,PLAYER_1.4,PLAYER_1.5,PLAYER_2.1,PLAYER_2.2,PLAYER_2.3,PLAYER_2.4,PLAYER_2.5,SCORE_1,SCORE_2,WINNER
0,22400061,0,0,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday,0,0,BOS
1,22400061,0,0,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday,0,0,BOS
2,22400061,3,12,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday,0,3,BOS
3,22400061,1,33,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday,2,3,BOS
4,22400061,1,44,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday,2,3,BOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124987,22400039,29,2854,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones,98,127,PHX
124988,22400039,29,2863,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones,98,127,PHX
124989,22400039,28,2863,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones,99,127,PHX
124990,22400039,27,2863,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones,100,127,PHX


In [11]:
data.loc[data['PERIOD'] > 1,'TIME_REMAINING'] += 720*(data['PERIOD']-1)
data

,GAME_ID,SCOREMARGIN,TIME_REMAINING,PERIOD,TEAM_1,TEAM_2,PLAYER_1.1,PLAYER_1.2,PLAYER_1.3,PLAYER_1.4,PLAYER_1.5,PLAYER_2.1,PLAYER_2.2,PLAYER_2.3,PLAYER_2.4,PLAYER_2.5,SCORE_1,SCORE_2
0,22400061,0,0,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday,0,0
1,22400061,0,0,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday,0,0
2,22400061,3,12,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday,0,3
3,22400061,1,33,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday,2,3
4,22400061,1,44,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124987,22400039,29,2854,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones,98,127
124988,22400039,29,2863,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones,98,127
124989,22400039,28,2863,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones,99,127
124990,22400039,27,2863,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones,100,127


In [ ]:
stats = pd.read_csv('stats_in_game.csv', index_col=0)
stats

In [21]:
data.to_csv('final_data.csv')


In [22]:
data = pd.read_csv('final_data.csv', index_col=0)
data

,GAME_ID,SCOREMARGIN,TIME_REMAINING,PERIOD,TEAM_1,TEAM_2,PLAYER_1.1,PLAYER_1.2,PLAYER_1.3,PLAYER_1.4,PLAYER_1.5,PLAYER_2.1,PLAYER_2.2,PLAYER_2.3,PLAYER_2.4,PLAYER_2.5,SCORE_1,SCORE_2,WINNER
0,22400061,0,0,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday,0,0,BOS
1,22400061,0,0,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday,0,0,BOS
2,22400061,3,12,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday,0,3,BOS
3,22400061,1,33,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday,2,3,BOS
4,22400061,1,44,1,NYK,BOS,OG Anunoby,Josh Hart,Karl-Anthony Towns,Mikal Bridges,Jalen Brunson,Jaylen Brown,Jayson Tatum,Al Horford,Derrick White,Jrue Holiday,2,3,BOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124987,22400039,29,2854,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones,98,127,PHX
124988,22400039,29,2863,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones,98,127,PHX
124989,22400039,28,2863,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones,99,127,PHX
124990,22400039,27,2863,4,LAL,PHX,Rui Hachimura,LeBron James,Anthony Davis,Cam Reddish,Austin Reaves,Bradley Beal,Kevin Durant,Jusuf Nurkić,Devin Booker,Tyus Jones,100,127,PHX
